In [1]:
import pandas as pd

# load aggregated table
df = pd.read_csv("DocumentedInfectionbyAgeGroup.csv")
df_clean = df.iloc[1:].copy()

df_clean.columns = [
    "agegroup",
    "admissions",
    "cai_count", "cai_pct",
    "hai_count", "hai_pct",
    "hbci_count", "hbci_pct",
    "nic_count", "nic_pct"
]

# convert numeric columns
num_cols = ["admissions","cai_count","hai_count","hbci_count","nic_count"]
for col in num_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce").fillna(0).astype(int)

# coarse age grouping
coarse_map = {
    "0–29 days": "<1",
    "1 to 11 months": "<1",
    "1 to 5 years": "1-5",
    "6 to 10 years": "6-10",
    "11 to 15 years": "11-15",
    "16 to 20 years": "16-20",
    "21 to 25 years": "21-25",
    "26 to 30 years": "26-30",
    "31 to 35 years": "31-35",
    "36 to 40 years": "36-40",
    "41 to 45 years": "41-45",
    "46 to 50 years": "46-50",
    "51 to 55 years": "51-55",
    "56 to 60 years": "56-60",
    "61 to 65 years": "61-65",
    "66 to 70 years": "66+",
    "71 to 75 years": "66+",
    "76 to 80 years": "66+",
    "81 to 85 years": "66+",
    "86 to 90 years": "66+",
    "91 to 95 years": "66+",
    "96+ years": "66+"
}

df_clean["coarse_age"] = df_clean["agegroup"].map(coarse_map)

grouped = df_clean.groupby("coarse_age", as_index=False).agg({
    "admissions":"sum",
    "cai_count":"sum",
    "hai_count":"sum",
    "hbci_count":"sum",
    "nic_count":"sum"
})

# lossless reconstruction
rows = []
for _, row in grouped.iterrows():
    age = row["coarse_age"]
    a = row["admissions"]
    c = row["cai_count"]
    h = row["hai_count"]
    b = row["hbci_count"]
    nic = row["nic_count"]
    none = a - (c + h + b + nic)
    rows += [{"agegroup": age, "infectionstatus": "cai"}] * c
    rows += [{"agegroup": age, "infectionstatus": "hai"}] * h
    rows += [{"agegroup": age, "infectionstatus": "hbci"}] * b
    rows += [{"agegroup": age, "infectionstatus": "nic"}] * nic
    rows += [{"agegroup": age, "infectionstatus": "none"}] * none

reconstructed_df = pd.DataFrame(rows)

# load antimicrobial classes
classes = pd.read_csv("AntibioticClassesAcrossHealthFacilities.csv")

def extract_count(x):
    s = str(x)
    parts = s.split()
    num = ''.join(ch for ch in parts[0] if ch.isdigit())
    return int(num) if num.isdigit() else 0

classes["count"] = classes["n = 982 (%)"].apply(extract_count)

total_prescriptions = 711
n_patients = 982

classes["proportion"] = classes["count"] / total_prescriptions
M = total_prescriptions / n_patients
classes["expected_per_patient"] = classes["proportion"] * M

expected_long = classes[["Class", "expected_per_patient"]].copy()

# cross join to attach expected-class demand to each patient
reconstructed_with_class = (
    reconstructed_df.assign(key=1)
    .merge(expected_long.assign(key=1), on="key")
    .drop(columns="key")
)

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# building glm-ready dataset
df_glm = reconstructed_with_class.copy()

# dummy coding
age_dummies = pd.get_dummies(df_glm["agegroup"], prefix="age", drop_first=True)
inf_dummies = pd.get_dummies(df_glm["infectionstatus"], prefix="inf", drop_first=True)

# clean illegal characters in dummy names
age_dummies.columns = (
    age_dummies.columns
    .str.replace("-", "_", regex=False)
    .str.replace("<", "lt", regex=False)
    .str.replace(">", "gt", regex=False)
)

inf_dummies.columns = (
    inf_dummies.columns
    .str.replace("-", "_", regex=False)
    .str.replace("<", "lt", regex=False)
    .str.replace(">", "gt", regex=False)
)

# combine dummy variables with original dataframe
df_glm = pd.concat([df_glm, age_dummies, inf_dummies], axis=1)

# define dependent variable
df_glm["y"] = df_glm["expected_per_patient"]

# record predictor columns
predictor_cols = list(age_dummies.columns) + list(inf_dummies.columns)
rhs = " + ".join(predictor_cols)

# create numeric-only modeling dataframe
df_model = df_glm.drop(columns=["agegroup", "infectionstatus", "Class"])

# fit negative binomial glm per antimicrobial class
models = {}
params = {}
mu_predictions = {}

for class_name in df_glm["Class"].unique():
    
    # subset for this class
    sub = df_glm[df_glm["Class"] == class_name].copy()
    
    # remove categorical columns before modeling
    sub_model = sub.drop(columns=["agegroup", "infectionstatus", "Class"])
    
    # formula using only numeric predictors
    formula = "y ~ " + rhs
    
    # fit glm
    model = smf.glm(
        formula=formula,
        data=sub_model,
        family=sm.families.NegativeBinomial()
    ).fit()
    
    models[class_name] = model
    params[class_name] = model.params
    mu_predictions[class_name] = model.predict(sub_model)

# assemble predictions into a tidy dataframe
prediction_frames = []

for class_name in df_glm["Class"].unique():
    sub = df_glm[df_glm["Class"] == class_name].copy()
    sub_model = sub.drop(columns=["agegroup", "infectionstatus", "Class"])
    sub["mu"] = mu_predictions[class_name]
    prediction_frames.append(sub[["agegroup", "infectionstatus", "Class", "mu"]])

glm_output = pd.concat(prediction_frames, ignore_index=True)


/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not 

In [2]:
def estimate_antimicrobial_demand(pop_age_df, by=["agegroup", "Class"]):
    """
    Estimate antimicrobial demand by age group and class using negative binomial GLM.
    Args:
        pop_age_df (pd.DataFrame): Population breakdown DataFrame (must include age group column).
        by (list): Columns to group output by (default: ["agegroup", "Class"]).
    Returns:
        pd.DataFrame: DataFrame with demand estimates per group.
    """
    # --- Load and prepare infection data ---
    df = pd.read_csv("DocumentedInfectionbyAgeGroup.csv")
    df_clean = df.iloc[1:].copy()
    df_clean.columns = [
        "agegroup", "admissions", "cai_count", "cai_pct", "hai_count", "hai_pct", "hbci_count", "hbci_pct", "nic_count", "nic_pct"
    ]
    num_cols = ["admissions","cai_count","hai_count","hbci_count","nic_count"]
    for col in num_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce").fillna(0).astype(int)
    coarse_map = {
        "0–29 days": "<1", "1 to 11 months": "<1", "1 to 5 years": "1-5", "6 to 10 years": "6-10", "11 to 15 years": "11-15",
        "16 to 20 years": "16-20", "21 to 25 years": "21-25", "26 to 30 years": "26-30", "31 to 35 years": "31-35",
        "36 to 40 years": "36-40", "41 to 45 years": "41-45", "46 to 50 years": "46-50", "51 to 55 years": "51-55",
        "56 to 60 years": "56-60", "61 to 65 years": "61-65", "66 to 70 years": "66+", "71 to 75 years": "66+",
        "76 to 80 years": "66+", "81 to 85 years": "66+", "86 to 90 years": "66+", "91 to 95 years": "66+", "96+ years": "66+"
    }
    df_clean["coarse_age"] = df_clean["agegroup"].map(coarse_map)
    grouped = df_clean.groupby("coarse_age", as_index=False).agg({
        "admissions":"sum", "cai_count":"sum", "hai_count":"sum", "hbci_count":"sum", "nic_count":"sum"
    })
    rows = []
    for _, row in grouped.iterrows():
        age = row["coarse_age"]
        a = row["admissions"]
        c = row["cai_count"]
        h = row["hai_count"]
        b = row["hbci_count"]
        nic = row["nic_count"]
        none = a - (c + h + b + nic)
        rows += [{"agegroup": age, "infectionstatus": "cai"}] * c
        rows += [{"agegroup": age, "infectionstatus": "hai"}] * h
        rows += [{"agegroup": age, "infectionstatus": "hbci"}] * b
        rows += [{"agegroup": age, "infectionstatus": "nic"}] * nic
        rows += [{"agegroup": age, "infectionstatus": "none"}] * none
    reconstructed_df = pd.DataFrame(rows)
    # --- Load antimicrobial classes ---
    classes = pd.read_csv("AntibioticClassesAcrossHealthFacilities.csv")
    def extract_count(x):
        s = str(x)
        parts = s.split()
        num = ''.join(ch for ch in parts[0] if ch.isdigit())
        return int(num) if num.isdigit() else 0
    classes["count"] = classes["n = 982 (%)"].apply(extract_count)
    total_prescriptions = 711
    n_patients = 982
    classes["proportion"] = classes["count"] / total_prescriptions
    M = total_prescriptions / n_patients
    classes["expected_per_patient"] = classes["proportion"] * M
    expected_long = classes[["Class", "expected_per_patient"]].copy()
    reconstructed_with_class = (
        reconstructed_df.assign(key=1).merge(expected_long.assign(key=1), on="key").drop(columns="key")
    )
    # --- GLM setup ---
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    df_glm = reconstructed_with_class.copy()
    age_dummies = pd.get_dummies(df_glm["agegroup"], prefix="age", drop_first=True)
    inf_dummies = pd.get_dummies(df_glm["infectionstatus"], prefix="inf", drop_first=True)
    age_dummies.columns = (
        age_dummies.columns.str.replace("-", "_", regex=False).str.replace("<", "lt", regex=False).str.replace(">", "gt", regex=False)
    )
    inf_dummies.columns = (
        inf_dummies.columns.str.replace("-", "_", regex=False).str.replace("<", "lt", regex=False).str.replace(">", "gt", regex=False)
    )
    df_glm = pd.concat([df_glm, age_dummies, inf_dummies], axis=1)
    df_glm["y"] = df_glm["expected_per_patient"]
    predictor_cols = list(age_dummies.columns) + list(inf_dummies.columns)
    rhs = " + ".join(predictor_cols)
    df_model = df_glm.drop(columns=["agegroup", "infectionstatus", "Class"])
    # --- Fit negative binomial GLM per antimicrobial class ---
    models = {}
    mu_predictions = {}
    for class_name in df_glm["Class"].unique():
        sub = df_glm[df_glm["Class"] == class_name].copy()
        sub_model = sub.drop(columns=["agegroup", "infectionstatus", "Class"])
        formula = "y ~ " + rhs
        model = smf.glm(formula=formula, data=sub_model, family=sm.families.NegativeBinomial()).fit()
        models[class_name] = model
        mu_predictions[class_name] = model.predict(sub_model)
    # --- Assemble predictions ---
    prediction_frames = []
    for class_name in df_glm["Class"].unique():
        sub = df_glm[df_glm["Class"] == class_name].copy()
        sub_model = sub.drop(columns=["agegroup", "infectionstatus", "Class"])
        sub["mu"] = mu_predictions[class_name]
        prediction_frames.append(sub[["agegroup", "infectionstatus", "Class", "mu"]])
    glm_output = pd.concat(prediction_frames, ignore_index=True)
    # --- Merge with population breakdown ---
    # If pop_age_df has 'agegroup' column, aggregate demand by age group and class
    if "agegroup" in pop_age_df.columns:
        pop_counts = pop_age_df.groupby("agegroup").agg({"Population": "sum"}).reset_index()
        glm_output = glm_output.merge(pop_counts, on="agegroup", how="left")
        glm_output["demand_estimate"] = glm_output["mu"] * glm_output["Population"]
    # Group and return
    return glm_output.groupby(by)["demand_estimate"].sum().reset_index()